<h2>Import functions and libraries:

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

# Sorting fuctions
def B_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if b[job[j]] < b[job[j+1]]:
                job[j], job[j+1] = job[j+1], job[j]
                
def R_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) < (b[job[j+1]]/p[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]
                
def RB_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) < (b[job[j+1]]/p[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]        
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) == (b[job[j+1]]/p[job[j+1]]) and (b[job[j]] < b[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]
                
def M_bubbleSort(bm, machine):
    for i in M:
        for j in range(0, m-i-1):
            if bm[machine[j]] > bm[machine[j+1]]:
                machine[j], machine[j+1] = machine[j+1], machine[j]

In [2]:
HBF_result = []
HRF_result = []
HRBF_result = []
for i in range(1, 21):
    
    # input
    name = "in" + str(i).zfill(2) + ".txt"

    f = open(name, 'r')
    line1 = f.readline()
    line2 = f.readline()
    line3 = f.readline()

    array = line1.split( )
    array = list(map(int, array))
    
    m = array[0]
    n = array[1]
    K = array[2]

    b = line2.split( )
    b = list(map(int, b))

    p = line3.split( )
    p = list(map(int, p))

    N = range(n)
    M = range(m)
    f.close()
 
    # Linear Relaxation
    # add var
    p3_1 = Model("problem3-1")    # build a new model

    x = p3_1.addVars(M, N, lb = 0, ub = 1, vtype = GRB.CONTINUOUS, name = "x")
    y = p3_1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "y")

    # setting the objective function 
    p3_1.setObjective(
        y
        , GRB.MAXIMIZE) 

    # add constraints and name them
    for i in M:
        p3_1.addConstr(y <= (quicksum(x[i, j] * b[j] for j in N)), name = f"min benefit")
    
    for j in N:
        p3_1.addConstr(quicksum(x[i, j] for i in M) <= 1, name = f"distribute to no more than 1 machine")
    
    for i in M:
        p3_1.addConstr(quicksum(x[i, j] * p[j] for j in N) <= K, name = f"satisfy capacity")
    
    p3_1.optimize()

    LR_ov = p3_1.objVal

    #HBF
    # Solve
    # Step 1: Sorting
    job_sorted = []
    for i in N:
        job_sorted.append(i)

    B_bubbleSort(b, job_sorted)
    
    # Step 2: Scheduling
    job_to_machine = [0] * n
    Pm = [0] * m
    Bm = [0] * m

    for i in job_sorted:
        # sort machine in priority
        machine_sorted = []
        for k in M:
            machine_sorted.append(k)
        M_bubbleSort(Bm, machine_sorted)
    
        # schedule jods to machine
        scheduled = False
        for j in machine_sorted:
            if (Pm[j] + p[i] <= K):
                job_to_machine[i] = j
                Pm[j] += p[i]
                Bm[j] += b[i]
                scheduled = True
                break         
        if (scheduled == False):
            job_to_machine[i] = -1
        
    # print results 
    HBF_ov = min(Bm)
    HBF_result.append((LR_ov - HBF_ov) / LR_ov)

    #HRF
    # Solve
    # Step 1: Sorting
    job_sorted = []
    for i in N:
        job_sorted.append(i)

    R_bubbleSort(b, job_sorted)
    
    # Step 2: Scheduling
    job_to_machine = [0] * n
    Pm = [0] * m
    Bm = [0] * m

    for i in job_sorted:
        # sort machine in priority
        machine_sorted = []
        for k in M:
            machine_sorted.append(k)
        M_bubbleSort(Bm, machine_sorted)
    
        # schedule jods to machine
        scheduled = False
        for j in machine_sorted:
            if (Pm[j] + p[i] <= K):
                job_to_machine[i] = j
                Pm[j] += p[i]
                Bm[j] += b[i]
                scheduled = True
                break         
        if (scheduled == False):
            job_to_machine[i] = -1
        
    # print results 
    HRF_ov = min(Bm)
    HRF_result.append((LR_ov - HRF_ov) / LR_ov)

    #HRBF
    # Solve
    # Step 1: Sorting
    job_sorted = []
    for i in N:
        job_sorted.append(i)

    RB_bubbleSort(b, job_sorted)
    
    # Step 2: Scheduling
    job_to_machine = [0] * n
    Pm = [0] * m
    Bm = [0] * m

    for i in job_sorted:
        # sort machine in priority
        machine_sorted = []
        for k in M:
            machine_sorted.append(k)
        M_bubbleSort(Bm, machine_sorted)
    
        # schedule jods to machine
        scheduled = False
        for j in machine_sorted:
            if (Pm[j] + p[i] <= K):
                job_to_machine[i] = j
                Pm[j] += p[i]
                Bm[j] += b[i]
                scheduled = True
                break         
        if (scheduled == False):
            job_to_machine[i] = -1
        
    # print results 
    HRBF_ov = min(Bm)
    HRBF_result.append((LR_ov - HRBF_ov) / LR_ov)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 51 rows, 136 columns and 408 nonzeros
Model fingerprint: 0xe9d5af28
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 51 rows, 136 columns, 408 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   3.000000e+30   8.000000e+00      0s
      73    1.4670000e+02   0.000000e+00   0.000000e+00      0s

Solved in 73 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.467000000e+02
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model

In [8]:
HBF_result = np.array(HBF_result)
HRF_result = np.array(HRF_result)
HRBF_result = np.array(HRBF_result)

print(HBF_result.mean())
print(HRF_result.mean())
print(HRBF_result.mean())

HBF_result_round = np.round(HBF_result, 4)
HRF_result_round = np.round(HRF_result, 4)
HRBF_result_round = np.round(HRBF_result, 4)

display(HBF_result_round)
display(HRF_result_round)
display(HRBF_result_round)

print(HBF_result_round.mean())
print(HRF_result_round.mean())
print(HRBF_result_round.mean())

for i in range(20):
    print(i+1, "&", HBF_result_round[i], "&", HRF_result_round[i], "&", HRBF_result_round[i], "\\\\\\hline")

0.07780333783677043
0.05814865611454632
0.057625476535771124


array([0.0184, 0.0659, 0.0863, 0.058 , 0.1809, 0.0655, 0.0949, 0.0859,
       0.0671, 0.0101, 0.1718, 0.0565, 0.0756, 0.1069, 0.0686, 0.0583,
       0.0116, 0.1332, 0.0387, 0.102 ])

array([0.0184, 0.0344, 0.068 , 0.0303, 0.0806, 0.0396, 0.0353, 0.0728,
       0.0671, 0.0173, 0.0535, 0.0307, 0.1092, 0.0611, 0.0599, 0.0743,
       0.0665, 0.0648, 0.1099, 0.0694])

array([0.0116, 0.0344, 0.068 , 0.0303, 0.0806, 0.0309, 0.0472, 0.0728,
       0.0671, 0.0173, 0.0535, 0.0242, 0.1261, 0.0611, 0.0427, 0.0743,
       0.0665, 0.0648, 0.1099, 0.0694])

0.07781
0.058155
0.05763499999999999
1 & 0.0184 & 0.0184 & 0.0116 \\\hline
2 & 0.0659 & 0.0344 & 0.0344 \\\hline
3 & 0.0863 & 0.068 & 0.068 \\\hline
4 & 0.058 & 0.0303 & 0.0303 \\\hline
5 & 0.1809 & 0.0806 & 0.0806 \\\hline
6 & 0.0655 & 0.0396 & 0.0309 \\\hline
7 & 0.0949 & 0.0353 & 0.0472 \\\hline
8 & 0.0859 & 0.0728 & 0.0728 \\\hline
9 & 0.0671 & 0.0671 & 0.0671 \\\hline
10 & 0.0101 & 0.0173 & 0.0173 \\\hline
11 & 0.1718 & 0.0535 & 0.0535 \\\hline
12 & 0.0565 & 0.0307 & 0.0242 \\\hline
13 & 0.0756 & 0.1092 & 0.1261 \\\hline
14 & 0.1069 & 0.0611 & 0.0611 \\\hline
15 & 0.0686 & 0.0599 & 0.0427 \\\hline
16 & 0.0583 & 0.0743 & 0.0743 \\\hline
17 & 0.0116 & 0.0665 & 0.0665 \\\hline
18 & 0.1332 & 0.0648 & 0.0648 \\\hline
19 & 0.0387 & 0.1099 & 0.1099 \\\hline
20 & 0.102 & 0.0694 & 0.0694 \\\hline
